In [1]:
import os
import tifffile
import numpy as np
import pandas as pd
import tensorflow as tf

from segmentation_models import Unet, get_preprocessing
from segmentation_models.losses import bce_dice_loss
from segmentation_models.metrics import f1_score

print("Number GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Segmentation Models: using `keras` framework.
Number GPUs Available:  1


In [2]:
### Model Parameters ###

MODEL = Unet
BACKBONE = "seresnet50"
EPOCH = 100

OPTIMIZER = "Adam"
METRICS = [f1_score]
LOSS = bce_dice_loss
MONITOR = "val_f1-score"

BATCH_SIZE = 16
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3

MODEL_NAME = f"{MODEL.__name__}_{BACKBONE}_{EPOCH}epoch_{BATCH_SIZE}batch_TrueOP"
os.makedirs(f'{MODEL_NAME}\\Results')

In [ ]:
# #Generating List of True Ortophotos for training
# trueop_train = open("train_tiles_trueop_manual.txt", "w")
# trueop_train.write("id\n")
# for path, subdirs, _ in os.walk(r'new_test_data\\trueop\\train'):
#     for dir in _:
#         if dir.endswith(".tif"):
#             fpath = os.path.join(path, dir)
#             trueop_train.write(fpath + "\n")
# trueop_train.close()

# # Generating List of nDSM for training
# ndsm_train = open("train_tiles_ndsm_manual.txt", "w")
# ndsm_train.write("id\n")
# for path, subdirs, _ in os.walk(r'new_test_data\\ndsm\\train'):
#     for dir in _:
#         if dir.endswith(".tif"):
#             fpath = os.path.join(path, dir)
#             ndsm_train.write(fpath + "\n")
# ndsm_train.close()
    
# # Generating List of masks for training
# mask_train = open("train_tiles_mask_manual.txt", "w")
# mask_train.write("id\n")
# for path, subdirs, _ in os.walk(r'new_test_data\\mask\\train'):
#     for dir in _:
#         if dir.endswith(".tif"):
#             fpath = os.path.join(path, dir)
#             mask_train.write(fpath + "\n")
# mask_train.close()

# # TEST ###################33
# trueop_test = open("test_tiles_trueop_manual.txt", "w")
# trueop_test.write("id\n")
# for path, subdirs, _ in os.walk(r'new_test_data\\trueop\\test'):
#     for dir in _:
#         if dir.endswith(".tif"):
#             fpath = os.path.join(path, dir)
#             trueop_test.write(fpath + "\n")
# trueop_test.close()

# # Generating List of nDSM for training
# ndsm_test = open("test_tiles_ndsm_manual.txt", "w")
# ndsm_test.write("id\n")
# for path, subdirs, _ in os.walk(r'new_test_data\\ndsm\\test'):
#     for dir in _:
#         if dir.endswith(".tif"):
#             fpath = os.path.join(path, dir)
#             ndsm_test.write(fpath + "\n")
# ndsm_test.close()
    
# # Generating List of masks for training
# mask_test = open("test_tiles_mask_manual.txt", "w")
# mask_test.write("id\n")
# for path, subdirs, _ in os.walk(r'new_test_data\\mask\\test'):
#     for dir in _:
#         if dir.endswith(".tif"):
#             fpath = os.path.join(path, dir)
#             mask_test.write(fpath + "\n")
# mask_test.close()


In [ ]:
#Generating List of True Ortophotos for training
trueop_train = open("new_train_tiles_trueop_manual.txt", "w")
trueop_train.write("id\n")
for path, subdirs, _ in os.walk(r'new_test_data\\trueop\\train'):
    for dir in _:
        if dir.endswith(".tif"):
            fpath = os.path.join(path, dir)
            trueop_train.write(fpath + "\n")
trueop_train.close()

# # Generating List of nDSM for training
# ndsm_train = open("new_train_tiles_ndsm_manual.txt", "w")
# ndsm_train.write("id\n")
# for path, subdirs, _ in os.walk(r'new_test_data\\ndsm\\train'):
#     for dir in _:
#         if dir.endswith(".tif"):
#             fpath = os.path.join(path, dir)
#             ndsm_train.write(fpath + "\n")
# ndsm_train.close()
    
# Generating List of masks for training
mask_train = open("new_train_tiles_mask_manual.txt", "w")
mask_train.write("id\n")
for path, subdirs, _ in os.walk(r'new_test_data\\mask\\train'):
    for dir in _:
        if dir.endswith(".tif"):
            fpath = os.path.join(path, dir)
            mask_train.write(fpath + "\n")
mask_train.close()


# TEST ###################33
trueop_test = open("new_test_tiles_trueop_manual.txt", "w")
trueop_test.write("id\n")
for path, subdirs, _ in os.walk(r'new_test_data\\trueop\\test'):
    for dir in _:
        if dir.endswith(".tif"):
            fpath = os.path.join(path, dir)
            trueop_test.write(fpath + "\n")
trueop_test.close()

# # Generating List of nDSM for training
# ndsm_test = open("new_test_tiles_ndsm_manual.txt", "w")
# ndsm_test.write("id\n")
# for path, subdirs, _ in os.walk(r'new_test_data\\ndsm\\test'):
#     for dir in _:
#         if dir.endswith(".tif"):
#             fpath = os.path.join(path, dir)
#             ndsm_test.write(fpath + "\n")
# ndsm_test.close()
    
# Generating List of masks for training
mask_test = open("new_test_tiles_mask_manual.txt", "w")
mask_test.write("id\n")
for path, subdirs, _ in os.walk(r'new_test_data\\mask\\test'):
    for dir in _:
        if dir.endswith(".tif"):
            fpath = os.path.join(path, dir)
            mask_test.write(fpath + "\n")
mask_test.close()

In [ ]:
train_data_trueop = pd.read_csv("new_train_tiles_trueop_manual.txt")
train_data_mask = pd.read_csv("new_train_tiles_mask_manual.txt")

test_data_trueop = pd.read_csv("new_test_tiles_trueop_manual.txt")
test_data_mask = pd.read_csv("new_test_tiles_mask_manual.txt")


In [ ]:
X_train = np.zeros((len(train_data_trueop), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
Y_train = np.zeros((len(train_data_trueop), IMG_HEIGHT, IMG_WIDTH, 1))

for i in range(len(train_data_trueop)):
    cache = np.zeros((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    truop_img= tifffile.imread(train_data_trueop["id"][i])
    mask_img = tifffile.imread(train_data_mask["id"][i])
    #ndsm_img = tifffile.imread(train_data_nDSM["id"][i])
    #ndsm_img[np.where(ndsm_img==-32767)] = 0 
    cache[:,:, :] = truop_img
    #cache[:,:, 3] = ndsm_img    
    X_train[i] = cache
    mask = mask_img[:,:,0]
    mask = np.expand_dims(mask, axis=-1)
    Y_train[i] = mask

In [ ]:
X_test = np.zeros((len(test_data_trueop), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
Y_test= np.zeros((len(test_data_trueop), IMG_HEIGHT, IMG_WIDTH, 1))

for i in range(len(test_data_trueop)):
    cache = np.zeros((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    truop_img= tifffile.imread(test_data_trueop["id"][i])
    mask_img = tifffile.imread(test_data_mask["id"][i])
    #ndsm_img = tifffile.imread(test_data_nDSM["id"][i])
    #ndsm_img[np.where(ndsm_img==-32767)] = 0 
    cache[:,:, :] = truop_img
    #cache[:,:, 3] = ndsm_img    
    X_test[i] = cache
    mask = mask_img[:,:,0]
    mask = np.expand_dims(mask, axis=-1)
    Y_test[i] = mask

In [ ]:
np.save("new_X_train_manual", X_train)
np.save("new_Y_train_manual", Y_train)
np.save("new_X_test_manual", X_test)
np.save("new_Y_test_manual", Y_test)

In [3]:
X_train = np.load("new_X_train_manual.npy")
Y_train = np.load("new_Y_train_manual.npy")
X_test= np.load("new_X_test_manual.npy")
Y_test = np.load("new_Y_test_manual.npy")

In [4]:
preprocess_input = get_preprocessing(BACKBONE)
from sklearn.model_selection import train_test_split
# load your data
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.111, random_state=42)

# preprocess input
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

In [5]:
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath=f'{MODEL_NAME}\\{MODEL_NAME}.h5', verbose=1, save_best_only=True, monitor=MONITOR, mode='max')
csv_logger = tf.keras.callbacks.CSVLogger(f'{MODEL_NAME}\\{MODEL_NAME}_log.csv', separator=',', append=False)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=1, monitor=MONITOR, mode='max')

In [6]:
# define model
model = MODEL(BACKBONE, input_shape=(None, None, IMG_CHANNELS), encoder_weights=None, classes=1, activation='sigmoid', decoder_block_type='transpose')
model.compile(OPTIMIZER, LOSS, METRICS)

# fit model
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCH,
    validation_data=(x_val, y_val),
    callbacks=[checkpointer, csv_logger] #Earlystopping removed
)

Epoch 1/100
286/286 [==============================] - 223s 635ms/step - loss: 0.6476 - f1-score: 0.6451 - val_loss: 1.2227 - val_f1-score: 0.7716

Epoch 00001: val_f1-score improved from -inf to 0.77165, saving model to Unet_seresnet50_100epoch_16batch_nDSM\Unet_seresnet50_100epoch_16batch_nDSM.h5


C:\Users\Mehmet\anaconda3\envs\tf25\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
286/286 [==============================] - 172s 602ms/step - loss: 0.3975 - f1-score: 0.7959 - val_loss: 0.3461 - val_f1-score: 0.8519

Epoch 00002: val_f1-score improved from 0.77165 to 0.85188, saving model to Unet_seresnet50_100epoch_16batch_nDSM\Unet_seresnet50_100epoch_16batch_nDSM.h5
Epoch 3/100
286/286 [==============================] - 171s 598ms/step - loss: 0.3304 - f1-score: 0.8311 - val_loss: 0.4385 - val_f1-score: 0.7961

Epoch 00003: val_f1-score did not improve from 0.85188
Epoch 4/100
286/286 [==============================] - 169s 591ms/step - loss: 0.3078 - f1-score: 0.8409 - val_loss: 0.2680 - val_f1-score: 0.8734

Epoch 00004: val_f1-score improved from 0.85188 to 0.87341, saving model to Unet_seresnet50_100epoch_16batch_nDSM\Unet_seresnet50_100epoch_16batch_nDSM.h5
Epoch 5/100
286/286 [==============================] - 169s 590ms/step - loss: 0.2670 - f1-score: 0.8622 - val_loss: 0.2751 - val_f1-score: 0.8637

Epoch 00005: val_f1-score did not improve f

In [7]:
from skimage.io import imsave
preds_val = model.predict(X_test, verbose=1)
preds_val_threshold = (preds_val > 0.5).astype(np.uint8)

i = -1
for path, subdirs, _ in os.walk(r'new_test_data\\mask\\test'):
    for dir in _:
        if dir.endswith(".tif"):
            i += 1
            img_pred = np.reshape(preds_val_threshold[i]*255, (256,256))
            imsave(f'{MODEL_NAME}\\Results\\{dir}', img_pred, check_contrast=False)
print("\nAll figures saved successfully.")

18/18 [==============================] - 12s 275ms/step

All figures saved successfully.
